In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Set the maximum number of rows to read at a time
chunksize = 1000

# Create an empty DataFrame to store the data
Similarity = pd.DataFrame()

# Loop over the CSV file and read in chunks of 1000 rows
for chunk in pd.read_csv('/content/merged_file.csv', chunksize=chunksize):
    # Append the current chunk to the DataFrame
    Similarity = Similarity.append(chunk, ignore_index=True)


    # Stop reading the file if we have read in the maximum number of rows
    if len(Similarity) >= 1000:
        break

# Enter the keywords to search
kw1 = input("Enter first keyword: ")
kw2 = input("Enter second keyword: ")

#def similarity_analysis(Similarity):
  
# Extract tweets for the first keyword
tweets_kw1 = Similarity.loc[Similarity['full_text'].str.contains(kw1, case=False)]
if len(tweets_kw1) == 0:
    print(f"No tweets found for '{kw1}'")
    exit()

# Extract tweets for the second keyword
tweets_kw2 = Similarity.loc[Similarity['full_text'].str.contains(kw2, case=False)]
if len(tweets_kw2) == 0:
    print(f"No tweets found for '{kw2}'")
    exit()

# Combine the tweets into two lists
docs1 = tweets_kw1['full_text'].tolist()
docs2 = tweets_kw2['full_text'].tolist()

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# fit the vectorizer on the documents
vectorizer.fit(docs1 + docs2)

# transform the documents into TF-IDF vectors
tfidf_matrix1 = vectorizer.transform(docs1)
tfidf_matrix2 = vectorizer.transform(docs2)

# compute the cosine similarity between the TF-IDF vectors
cosine_sim = cosine_similarity(tfidf_matrix1, tfidf_matrix2)

# plot the heatmap
fig, ax = plt.subplots(figsize=(25,10))
sns.heatmap(cosine_sim, cmap='YlOrBr') 
plt.title(f"Similarity Between Tweets having '{kw1}' and '{kw2}'")
plt.show()

# get the indices of the most similar tweets
idx = cosine_sim.argmax(axis=1)
